# Recoleccion de fotos desde el jetson nano

#### importamos librerias y creamos las instacias para la camara y el trailet para verla

In [1]:
import traitlets
import ipywidgets.widgets as widgets
import os
from uuid import uuid1
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

#### Creamos las carpetas para el dataset 

In [2]:
pieza_dir = 'dataset/pieza' #blocked
no_pieza_dir = 'dataset/no_pieza' #free

# por si la carpeta ya existe
try:
    os.makedirs(no_pieza_dir)
    os.makedirs(pieza_dir)
except FileExistsError:
    print('Las carpetas ya existen.')

#### Creamos los botones y los contadores para "pieza" y " no_pieza"

In [3]:
button_layout = widgets.Layout(width='128px', height='64px')
no_pieza_button = widgets.Button(description='Foto SIN pieza', button_style='success', layout=button_layout)
pieza_button = widgets.Button(description='Foto CON pieza', button_style='danger', layout=button_layout)
no_pieza_count = widgets.IntText(layout=button_layout, value=len(os.listdir(no_pieza_dir)))
pieza_count = widgets.IntText(layout=button_layout, value=len(os.listdir(pieza_dir)))

#### Definimos 3 funciones, una para que guarde las fotos y las otras dos para q las guarde dependiendo del boton que presionemos

In [4]:
def guardar_foto(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def guardar_no_pieza():
    global no_pieza_dir, no_pieza_count
    guardar_foto(no_pieza_dir)
    no_pieza_count.value = len(os.listdir(no_pieza_dir))
    
def guardar_pieza():
    global pieza_dir, pieza_count
    guardar_foto(pieza_dir)
    pieza_count.value = len(os.listdir(pieza_dir))
    
no_pieza_button.on_click(lambda x: guardar_no_pieza())
pieza_button.on_click(lambda x: guardar_pieza())

#### Mostramos la camara del robot y los botones para agregar fotos al dataset

In [5]:
display(image)
display(widgets.HBox([no_pieza_count, no_pieza_button]))
display(widgets.HBox([pieza_count, pieza_button]))

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

##### Para parar la conexion de la camara

In [7]:
camera.stop()

# Entrenamiento de la IA

In [2]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [3]:
dataset = datasets.ImageFolder(
    'dataset',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [4]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 5, 5])

In [5]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=True,
    num_workers=0
)

In [6]:
model = models.alexnet(pretrained=True)

In [7]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

In [8]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
NUM_EPOCHS = 10
MODEL_PATH = 'modelo_entrenado.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):
    
    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()
    
    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))
    
    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), MODEL_PATH)
        best_accuracy = test_accuracy

0: 0.400000
